## Блок 03 - Первые отчеты по API

In [52]:
import requests
import yaml
from pprint import pprint 

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.net/stat/v1/data'
startDate = '2022-01-01'
endDate = '2022-01-07'
counter = '53600281' #'21075004'
dimentions = ['ym:s:date']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews', 'ym:s:percentNewVisitors']
dimentions_string = ','.join(dimentions)
metrics_string = ','.join(metrics)
params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    'dimentions': dimentions_string,
    'metrics': metrics_string,
    'oauth_token': token
}

In [6]:
print(metrics_string)

ym:s:visits,ym:s:users,ym:s:pagwviews,ym:s:percentNewVisitors


In [7]:
print(dimentions_string)

ym:s:date


In [54]:
r = requests.get(API_URL, params= params)
data = r.json()

In [55]:
pprint(data)

{'contains_sensitive_data': False,
 'data': [{'dimensions': [], 'metrics': [5728.0, 5162.0, 7274.0, 92.03796978]}],
 'data_lag': 150,
 'max': [5728.0, 5162.0, 7274.0, 92.03796978],
 'min': [5728.0, 5162.0, 7274.0, 92.03796978],
 'query': {'adfox_event_id': '0',
           'attr_name': '',
           'attribution': 'LastSign',
           'auto_group_size': '1',
           'currency': 'RUB',
           'date1': '2022-01-01',
           'date2': '2022-01-07',
           'dimensions': [],
           'group': 'Week',
           'ids': [53600281],
           'limit': 100,
           'metrics': ['ym:s:visits',
                       'ym:s:users',
                       'ym:s:pageviews',
                       'ym:s:percentNewVisitors'],
           'offline_window': '21',
           'offset': 1,
           'quantile': '50',
           'sort': ['-ym:s:visits']},
 'sample_share': 1.0,
 'sample_size': 5730,
 'sample_space': 5730,
 'sampled': False,
 'total_rows': 1,
 'total_rows_rounded': False,


In [20]:
token

'AQAAAAABLzdZAAedbkj20oEZuUbvgLNxdyG3k_w'

In [34]:
print(requests.get(API_URL, params= params))

<Response [400]>
